In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter

%matplotlib inline

/home/samuel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Imports

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
#race = "hispanic"
#race = "white"
#race = "mixed"

In [ ]:
#X = np.load('../Data/' + race + '/X.npy')
#Y2 = np.load('../Data/' + race + '/Y2.npy')

In [9]:
X_hispanic = np.load('../Data/hispanic/X.npy')
Y_hispanic = np.load('../Data/hispanic/Y2.npy')
X_white = np.load('../Data/white/X.npy')
Y_white = np.load('../Data/white/Y2.npy')
X_mixed = np.load('../Data/mixed/X.npy')
Y_mixed = np.load('../Data/mixed/Y2.npy')

In [10]:
size_hispanic=37575

In [11]:
def split_train_test(X, Y):
    # shuffle
    np.random.seed(97)
    idx = np.random.permutation(len(X))
    X = X[idx]
    Y = Y[idx]
    
    X = X[:size_hispanic]
    Y = Y[:size_hispanic]

    # split into training and test sets
    TEST_SET_SIZE = int(0.1*len(Y))
    X_train, X_test = X[:-TEST_SET_SIZE], X[-TEST_SET_SIZE:]
    Y_train, Y_test = Y[:-TEST_SET_SIZE].astype(int), Y[-TEST_SET_SIZE:].astype(int)
    return X_train, X_test, Y_train, Y_test

In [12]:
X_train_h, X_test_h, Y_train_h, Y_test_h = split_train_test(X_hispanic, Y_hispanic)
X_train_w, X_test_w, Y_train_w, Y_test_w = split_train_test(X_white, Y_white)
X_train_m, X_test_m, Y_train_m, Y_test_m = split_train_test(X_mixed, Y_mixed)

# Feature Scaling
Fit scaler based on training data, then transform both the training and test data.

In [ ]:
# feature scaling: scale features based on training data only
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def feature_scale(X_train, X_test):
    
    mm_scaler = MinMaxScaler(feature_range=(-1,1))
    X_train[:,-4:] = mm_scaler.fit_transform(X_train[:,-4:])
    X_test[:,-4:] = mm_scaler.transform(X_test[:,-4:])
    
    std_scaler = StandardScaler()
    X_train[:,:-4] = std_scaler.fit_transform(X_train[:,:-4])
    X_test[:,:-4] = std_scaler.transform(X_test[:,:-4])
    return X_train, X_test

X_train_h, X_test_h = feature_scale(X_train_h, X_test_h)
X_train_w, X_test_w = feature_scale(X_train_w, X_test_w)
X_train_m, X_test_m = feature_scale(X_train_m, X_test_m)

/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Results Function

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

def results(classifier, X_test, Y_test):
    Y_pred_test = classifier.predict(X_test)
    print("Test accuracy score: " + str(accuracy_score(Y_test.astype(int), Y_pred_test)))
    print("ROC: " + str(roc_auc_score(Y_test, classifier.predict_proba(X_test)[:,1])))
    #print("ROC: " + str(roc_auc_score(Y_test, Y_pred_test)))
    matrix = confusion_matrix(Y_test.astype(int), Y_pred_test)
    tn, fp, fn, tp = matrix.ravel()
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    g_mean = np.sqrt(sensitivity*specificity)
    print("PPV: " + str(ppv))
    print("NPV: " + str(npv))
    print("Sensitivity: " + str(sensitivity))
    print("Specificity: " + str(specificity))
    print("G-Mean: " + str(g_mean))
    print("Confusion matrix:\n" + str(matrix))
    

## Decision Tree

In [ ]:

# min_sample_split: 300,400
# min_samples_leaf: 200
# max_depth: 130
# min_weight_fraction_leaf: .01
param_grid = [{'max_depth':[40,50,60], 'min_samples_leaf':[250,260,270,280,290]}]
tree_clf_reg = DecisionTreeClassifier()
dt_grid_search = GridSearchCV(tree_clf_reg, param_grid, cv=3, scoring=make_scorer(roc_auc_score), verbose=3)
dt_grid_search.fit(X_train_h, Y_train_h.astype(int))

In [ ]:
cvres = dt_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(dt_grid_search.best_params_))

In [ ]:
results(dt_grid_search, X_test_h, Y_test_h)

In [ ]:
tree_clf_h = DecisionTreeClassifier(max_depth=130, min_samples_leaf=200)
tree_clf_h.fit(X_train_h, Y_train_h)

tree_clf_w = DecisionTreeClassifier(max_depth=130, min_samples_leaf=200)
tree_clf_w.fit(X_train_w, Y_train_w)

tree_clf_m = DecisionTreeClassifier(max_depth=130, min_samples_leaf=200)
tree_clf_m.fit(X_train_m, Y_train_m)

In [ ]:
results(tree_clf_h, X_test_h, Y_test_h)

In [ ]:
results(tree_clf_w, X_test_w, Y_test_w)

In [ ]:
results(tree_clf_m, X_test_m, Y_test_m)

In [ ]:
d_h = dict(enumerate(tree_clf_h.feature_importances_))
sorted(d_h, key=d_h.get, reverse=True)

In [ ]:
d_w = dict(enumerate(tree_clf_w.feature_importances_))
sorted(d_w, key=d_w.get, reverse=True)

## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, Y_train.astype(int))

In [ ]:
results(knn_clf)

## Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = [{'C':[.01, .1, .5]}]
lr_grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, 
                              scoring='accuracy', verbose=5
                             )
lr_grid_search.fit(X_train, Y_train.astype(int))

In [ ]:
cvres = lr_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(lr_grid_search.best_params_))

In [ ]:
results(lr_grid_search)

In [ ]:
lr = LogisticRegression(C=.05)
lr.fit(X_train, Y_train.astype(int))
results(lr)

## Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
        LogisticRegression(), n_estimators=40,
        max_samples=1000, bootstrap=True, 
        verbose=3
)

bag_clf.fit(X_train, Y_train)

In [ ]:
# probably add cross-validation?

In [ ]:
results(bag_clf)

## Random Forest

In [ ]:

#scoring=make_scorer(roc_auc_score)

param_grid = [{'max_features':[50,300], 'n_estimators':[10,20,30], 'min_samples_leaf':[200,300,400]}]
rf_grid_search = GridSearchCV(RandomForestClassifier(), 
            param_grid, cv=3, scoring=make_scorer(roc_auc_score), 
            verbose=5
)
rf_grid_search.fit(X_train_h, Y_train_h)

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=30, min_samples_leaf=200, max_features=300)
rf_clf.fit(X_train_m, Y_train_m)
results(rf_clf, X_test_m, Y_test_m)

In [ ]:
roc_auc_score(Y_test_m, rf_clf.predict_proba(X_test_m)[:,1])

In [ ]:
rf_clf.feature_importances_

In [ ]:
results(rf_grid_search, X_test_h, Y_test_h)

In [ ]:
cvres = rf_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(rf_grid_search.best_params_))

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, max_features=20, min_samples_leaf=50, verbose=3)
rf_clf.fit(X_train_m, Y_train_m)


In [ ]:
results(rf_clf, X_test_m, Y_test_m)

## Kernel SVM

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(kernel="rbf", gamma=5, C=0.1, verbose=3)
svm_clf.fit(X_train_h, Y_train_h)

In [ ]:
results(svm_clf, X_test_h, Y_test_h)

In [ ]:
mm_scaler = MinMaxScaler(feature_range=(-1,1)).fit(X_train_h)

## Linear SVM

In [ ]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1e-5, loss="hinge")
mm_scaler = MinMaxScaler(feature_range=(-1,1)).fit(X_train_h)
svm_clf.fit(mm_scaler.transform(X_train_h), Y_train_h)

In [ ]:
results(svm_clf, mm_scaler.transform(X_test_h), Y_test_h)